<p style="border:2px solid black"> </p>
<span style="font-family:Lucida Bright;">
<p style="margin-bottom:0.8cm"></p>
<center>
<font size="6"><b>Understanding Music Listening Habits</b></font>
<p style="margin-bottom:-0.1cm"></p>
<font size="6"><b>Using Large-scale Smartphone Data</b>  </font>
    
<p style="margin-bottom:0.5cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<p style="margin-bottom:1cm"></p>
<font size="5"><b>Data Cleaning</b></font>
<br>
<font size="3"><b></b></font>
</center>
<p style="margin-bottom:0.4cm"></p>
<p style="border:2px solid black"> </p>


# Initialization
<hr style="border:2px solid black"></hr>


In [1]:
%matplotlib inline

from toolbox.initialize import *

# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


ModuleNotFoundError: No module named 'toolbox'

# Load data
<hr style="border:2px solid black"></hr>

## Get the dataframes containing data in the raw format

In [ ]:
# Get the full dataset
data_root_raw, data_subfolders_raw, data_files_raw = t.get_paths_raw_data()
df_raw = t.load_data_from_files(data_root_raw, spark, method='avro')

# Get the sample data
path_df_sample_raw_1E6 = \
    Config.Path.project_data_root / 'df_sample_raw_1E6.parquet'

df_sample_raw_1E6 = t.load_data_from_files(path_df_sample_raw_1E6, spark)

## Choose dataframe

In [ ]:
df = df_raw

display_middle_results = False

# Cleaning
<hr style="border:2px solid black"></hr>


## Removing rows where activity has been deleted

In [3]:
df_clean = (
    df
    .where(f.col('deleted_time').isNull()
           | (f.col('deleted_time') == ''))
)

# Show the top rows of the resulting dataframe.
if display_middle_results:
    df_clean.limit(100).toPandas().head()


Execution time: 0.18348 s.


## Dropping duplicates of activity id (keeping the most recent)

In [4]:
# Define the time format used for the time stamp - strings in the
# raw database.
time_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

# Define a partitioning by activity id.
window_id = Window.partitionBy('id')

# Drop rows containing the duplicates of the activity ID.
# Keep the activity ID with the latest timestamp-
df_clean = (
    df_clean
    
    # Convert the start time to timestamp.
    .withColumn('start_time',
                f.to_timestamp('start_time', time_format)) 
    
    # Save the latest start time for each activity ID in a separate column.
    .withColumn('latest_start_time', f.max('start_time').over(window_id))
    
    # Keep only rows with the latest start time for the given activity ID.
    .where(f.col('start_time') == f.col('latest_start_time'))
        
    # Keep only the first occurrence of each activity ID.
    .dropDuplicates(['id'])
    
    # Delete the column containing the lates start time.
    .drop('latest_start_time')
)

# Show the top rows of the resulting dataframe.
if display_middle_results:
    df_clean.limit(100).toPandas().head()

Execution time: 0.12910 s.


## Exploding the columns `devices` and `tracks` and flattening the schema

In [5]:
# Explore the column devices and tracks
# and flatten the schema.
df_clean = t.format_dataframe(df_clean)

# Show the top rows of the resulting dataframe.
if display_middle_results:
    df_clean.limit(100).toPandas().head()

Formatting dataframe:
	Exploding columns containing lists.
	Flattening the dataframe schema.
	Execution time: 0.11851


## Replacing start and end time with start time and duration

In [6]:
# Define the time format used for the time stamp - strings in the
# raw database.
time_format = "yyyy-MM-dd'T'HH:mm:ss.SSSXXX"

df_clean = (
    df_clean
#     .withColumn('start_time',
#                 f.to_timestamp('start_time', time_format))
    .withColumn('end_time',
                f.to_timestamp('end_time', time_format))
    .withColumn('activity_duration',
                f.col('end_time').cast(LongType()) 
                - f.col('start_time').cast(LongType()))   
    .withColumn('tracks_start_time',
                f.to_timestamp('tracks_start_time', time_format))
    .withColumn('tracks_end_time',
                f.to_timestamp('tracks_end_time', time_format))
    .withColumn('track_duration',
                f.col('tracks_end_time').cast(LongType()) 
                - f.col('tracks_start_time').cast(LongType()))
    .drop('end_time')
    .drop('tracks_end_time')
)

# Show the top rows of the resulting dataframe.
if display_middle_results:
    df_clean.limit(100).toPandas().head()
    

Execution time: 0.09664 s.


## Merging Track ID and Track URI

I can't decide whether it's a good idea or not. `tracks_id` seems to be a path to a file in the Sony system, which I can't see give any useful information, maybe aside of comparing the track properties of multiple rows with the same track id to look for inconsitencies. I will be dropping the merge for now.

In [ ]:
# df_clean = (
#     df_clean
#     .withColumn('track_id',
#                 f.concat(f.col('tracks_id'),
#                          f.col('tracks_uri')))
#     .drop('tracks_id')
#     .drop('tracks_uri')
# )

# df_clean.limit(3).toPandas().head()

## Drop redundant columns

Let's remove the columns:

* `deleted_time`
* `devices_type`
* `yearmonth`

In [8]:
df_clean = (
    df_clean
    .drop('deleted_time')
    .drop('devices_type')
    .drop('yearmonth')
)

# Show the top rows of the resulting dataframe.
if display_middle_results:
    df_clean.limit(100).toPandas().head()

## Rename and organize columns

In [9]:
df_clean = (
    df_clean
    
    # Give columns meaningful names.
    .withColumnRenamed('id', 'activity_id')
    .withColumnRenamed('useruuid', 'user_id')
    .withColumnRenamed('start_time', 'activity_start_time')
    .withColumnRenamed('devices_name', 'device_name')
    .withColumnRenamed('devices_id', 'device_id')
    .withColumnRenamed('tracks_start_time', 'track_start_time')
    .withColumnRenamed('tracks_artist', 'track_artist')
    .withColumnRenamed('tracks_album', 'track_album')
    .withColumnRenamed('tracks_title', 'track_title')
    .withColumnRenamed('tracks_player', 'track_player')
    .withColumnRenamed('tracks_id', 'track_id')
    .withColumnRenamed('tracks_uri', 'track_uri')
    
    # Place the columns belonging to the same group next to each other.
    .select('user_id',
            'activity_id', 'activity_start_time', 'activity_duration',
            'device_id', 'device_name',
            'track_artist', 'track_title', 'track_album',
            'track_player', 'track_start_time', 'track_duration',
            'track_id', 'track_uri')
    
    # Sort the data.
    .orderBy(f.asc('user_id'),
             f.asc('activity_start_time'))
)

# Show the top rows of the resulting dataframe.
# df_clean.limit(100).toPandas().head()

Execution time: 0.10798 s.


## Replace undefined cell values with null

In [ ]:
# Define a function that will replace cells with empty
# or undefined values with null.
def replace_invalid_with_null(column_name):
    return (f.when(((f.col(column_name) == '')
                   | (f.col(column_name) == '<unknown>')
                   | (f.col(column_name).contains( '�'))),
                  None)
            .otherwise(f.col(column_name))
           )

# Define the expression thaw will be used together with the
# "select" statement to replace undefined values with null.
expression_replace_invalid_with_null = \
    [replace_invalid_with_null(column_name).alias(column_name)
     for column_name in df.columns]

# Execute the cleaning.
df_clean = df_clean.select(*expression_replace_invalid_with_null)

# Show the top rows of the resulting dataframe.
if display_middle_results:
    df_clean.limit(100).toPandas().head()

# Save the dataframe
<hr style="border:2px solid black"></hr>


In [10]:
# Save the start time for timing.
start_time = time.time()

# Save the dataframe with partitions defined by the first
# two letters of the user ID.
(df_clean
 .withColumn('user_id_prefix', f.col('user_id').substr(0, 2))
 .write.mode('overwrite')
 .partitionBy('user_id_prefix')
 .parquet(str(Config.Path.project_data_root / 'df_clean'))
)

# Print the execution time.
print(f'Execution time: {time.time() - start_time:.5f} s.')

Py4JJavaError: An error occurred while calling o185.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:178)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:175)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:213)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:210)
	at org.apac